## Uniprot Service
##### example: https://nbviewer.ipython.org/github/cokelaer/bioservices/blob/master/doc/notebooks/UniProt.ipynb

## Convert GenBank ID (Accession number) to UniProt ID (Accession number)

In [14]:
import requests

result_converter = lambda row: row.split("\t")[-1]

# Retrieve/ID mapping service from UniProt
url = 'https://www.uniprot.org/uploadlists/'

params = {
    'from': 'EMBL',
    'to': 'ACC',
    'format': 'tab',
    'query': 'ATE90961.1 YP_010084900.1'
}

r = requests.post(url, data=params)
print(r.text)
print([result_converter(i) for i in r.text.splitlines()[1:]])

From	To
ATE90961.1	A0A291B645

['A0A291B645']


In [32]:
from bioservices import UniProt

source = "ATE90961.1"

uniprot = UniProt(verbose=False)
result = uniprot.mapping("EMBL", "ACC", source)

In [33]:
result[source][0]

'A0A291B645'

## Read a sample protein sequence from FASTA file

In [9]:
from Bio import SeqIO
record = SeqIO.read("sample.fasta", format="fasta")

print("ID:", record.id)
print("Description:", record.description)
print("Sequence:", record.seq)

ID: sp|A0A2D2AKT0|A0A2D2AKT0_9VIRU
Description: sp|A0A2D2AKT0|A0A2D2AKT0_9VIRU Capsid protein alpha OS=Redspotted grouper nervous necrosis virus OX=43763 PE=3 SV=1
Sequence: MVRKGEKKLAKPATTKAANPQPRRRANNRRRSNRTDAPVSKASTVTGFGRGTNDVHLSGMSRISQAVLPAGTGTDGYVVVDATIVPDLLPRLGHAARIFQRYAVETLEFEIQPMCPANTGGGYVAGFLPNPTDNDHTFDAIQATRGAVVAKWWESRTVRPQYTRTLLWTSSGKEQRLTSPGRLILLCVGDNTDVVNVSVLCRWSVRLSVPSLETPEETTAPIMTQGSLYNDSLSTNDFKSILLGSIPLDIAPDGAVFQLDRPLSIDYSLGTGDVDRAVYWHLKKFAGNASTPAGWFRWGIWDNFNKTFTDGVAYYPDEQPRQILLPVGTVCTRVDSEN


In [13]:
from bioservices.apps.fasta import FASTA
record = FASTA()
record.read_fasta("sample.fasta")

print("ID:", record.identifier)
print("Accession:", record.accession)
print("Description:", record.header)
print("Sequence:", record.sequence)

Only sp and gi header are recognised so far but sequence and header are loaded
ID: tr|A0A2D2AKT0|A0A2D2AKT0_9VIRU
Accession: None
Description: tr|A0A2D2AKT0|A0A2D2AKT0_9VIRU Capsid protein alpha OS=Redspotted grouper nervous necrosis virus OX=43763 PE=3 SV=1
Sequence: MVRKGEKKLAKPATTKAANPQPRRRANNRRRSNRTDAPVSKASTVTGFGRGTNDVHLSGMSRISQAVLPAGTGTDGYVVVDATIVPDLLPRLGHAARIFQRYAVETLEFEIQPMCPANTGGGYVAGFLPNPTDNDHTFDAIQATRGAVVAKWWESRTVRPQYTRTLLWTSSGKEQRLTSPGRLILLCVGDNTDVVNVSVLCRWSVRLSVPSLETPEETTAPIMTQGSLYNDSLSTNDFKSILLGSIPLDIAPDGAVFQLDRPLSIDYSLGTGDVDRAVYWHLKKFAGNASTPAGWFRWGIWDNFNKTFTDGVAYYPDEQPRQILLPVGTVCTRVDSEN


## Check accession number

In [ ]:
import re

UNIPROT_ACCESSION_NUMBER_FORMAT = r"^([A-N,R-Z][0-9]([A-Z][A-Z,0-9][A-Z,0-9][0-9]){1,2})|([O,P,Q][0-9][A-Z,0-9][A-Z,0-9][A-Z,0-9][0-9])(\.\d+)?$"  # noqa: E501
GENBANK_ACCESSION_NUMBER_FORMAT = r"^([A-Za-z0-9]+\d+(\.\d+)?)|((N|X|W|A)P_\d+)$"


def is_accession_format(v: str) -> bool:
    return (
        re.fullmatch(GENBANK_ACCESSION_NUMBER_FORMAT, v) is not None
        or re.fullmatch(UNIPROT_ACCESSION_NUMBER_FORMAT, v) is not None
    )

is_accession_format("WP_123456")

True

## Uniprot Blast
##### Documentation: https://www.ebi.ac.uk/seqdb/confluence/pages/viewpage.action?pageId=94147939
##### NCBIblast example: https://nbviewer.ipython.org/github/cokelaer/bioservices/blob/master/doc/notebooks/NCBIBlast.ipynb

In [7]:
import json
from bioservices import NCBIblast


OUTPUT_FILE_NAME = "blast-uniprot-sample"

def blast(sequence_file_name: str, output_file_name: str):
    out = run(["python", "ncbiblast.py", "--outformat", "json", "--exp", "1e-3", "--gapalign", "false", "--email", "leaf.ying.work@gmail.com", "--stype", "protein", "--program", "blastp", "--database", "uniprotkb", "--sequence", sequence_file_name, "--outfile", output_file_name], capture_output=True)
    out.check_returncode()

def identify_sequence(source_file_name: str, threshold: float = 50.0):
    blast(source_file_name, OUTPUT_FILE_NAME)

    data = {}
    with open("./blast-uniprot-test.json.json", "r") as f:
        data = json.loads(f.read())

    # Get similar sequences
    sequences = []
    first_idendity = -1
    for sequence in data["hits"]:
        hsp_length = len(sequence["hit_hsps"])

        if (hsp_length > 0):
            identity = sum(hsp["hsp_identity"] for hsp in sequence["hit_hsps"]) / hsp_length
            if (identity < threshold):
                break

            if (first_idendity == -1):
                first_idendity = identity
                sequences.append(sequence)
            else:
                if (first_idendity == identity):
                    sequences.append(sequence)
                else:
                    break

    return first_idendity, sequences


threshold = 50.0

identity, sequences = identify_sequence("sample_only_seq.fasta", threshold)
print(f"Identity: {identity}")
print(f"Sequences: {sequences}")

Identity: 100.0
Sequences: [{'hit_num': 1, 'hit_def': 'TR:A0A249Y6I2 A0A249Y6I2_9VIRU Major capsid protein (Fragment) OS=Cherax quadricarinatus iridovirus OX=2035708 PE=3 SV=1', 'hit_db': 'TR', 'hit_id': 'A0A249Y6I2_9VIRU', 'hit_acc': 'A0A249Y6I2', 'hit_desc': 'Major capsid protein (Fragment) OS=Cherax quadricarinatus iridovirus OX=2035708 PE=3 SV=1', 'hit_url': 'https://www.uniprot.org/uniprot/A0A249Y6I2', 'hit_xref_url': 'https://www.ebi.ac.uk/ebisearch/search.ebi?db=uniprot&query=A0A249Y6I2', 'hit_uni_de': 'Major capsid protein (Fragment)', 'hit_uni_os': 'Cherax quadricarinatus iridovirus', 'hit_uni_ox': '2035708', 'hit_uni_pe': '3', 'hit_uni_sv': '1', 'hit_len': 477, 'hit_hsps': [{'hsp_num': 1, 'hsp_score': 2509, 'hsp_bit_score': 971.074, 'hsp_expect': 0.0, 'hsp_align_len': 477, 'hsp_identity': 100.0, 'hsp_positive': 100.0, 'hsp_gaps': 0, 'hsp_query_frame': '0', 'hsp_hit_frame': '0', 'hsp_strand': 'none/none', 'hsp_query_from': 1, 'hsp_query_to': 477, 'hsp_hit_from': 1, 'hsp_hit_to

## TODO: Identify rules

## Subprocess introduction
Documentation: https://docs.python.org/3.9/library/subprocess.html

In [8]:
import time, asyncio

def blocking_io():
    print(f"start blocking_io at {time.strftime('%X')}")
    # Note that time.sleep() can be replaced with any blocking
    # IO-bound operation, such as file operations.
    time.sleep(1)
    print(f"blocking_io complete at {time.strftime('%X')}")
    return True

async def main():
    print(f"started main at {time.strftime('%X')}")

    result = await asyncio.gather(asyncio.to_thread(blocking_io))

    print(f"finished main at {time.strftime('%X')}")

    return result


task = asyncio.create_task(main())
print("sease")

await asyncio.sleep(1)

print("sease")
print(await task)

sease
started main at 22:10:14
start blocking_io at 22:10:14
blocking_io complete at 22:10:15
sease
finished main at 22:10:15
[True]



## 延伸閱讀  
async-io-design-patterns-python: https://www.maxlist.xyz/2020/04/03/async-io-design-patterns-python/  
concurrency-programming: https://www.maxlist.xyz/2020/04/09/concurrency-programming/  
python-multithreading: https://www.maxlist.xyz/2020/03/15/python-threading/  
Concurrency: https://realpython.com/python-concurrency/

## NCBI Blast (Deprecated)

In [ ]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SeqIO
record = SeqIO.read("sample.fasta", format="fasta")
# entrez query builder: https://www.ncbi.nlm.nih.gov/protein/advanced
result = NCBIWWW.qblast("blastp", "swissprot", record.format("fasta"),
                        expect=0.01,
                        hitlist_size=5)
record = NCBIXML.read(result)
result.close()

In [ ]:
for description in record.descriptions:
    print(description.title)

print()
for alignment in record.alignments:
    print(alignment.hit_def)

sp|Q05815.2| RecName: Full=Major capsid protein; Short=MCP; AltName: Full=P50 [Invertebrate iridescent virus 6]
sp|O39163.1| RecName: Full=Major capsid protein; Short=MCP; AltName: Full=P50 [Wiseana iridescent virus]
sp|O39164.1| RecName: Full=Major capsid protein; Short=MCP; AltName: Full=P50 [Costelytra zealandica iridescent virus]
sp|Q197E6.1| RecName: Full=Major capsid protein; Short=MCP [Invertebrate iridescent virus 3]
sp|P22166.1| RecName: Full=Major capsid protein; Short=MCP; AltName: Full=P50 [Simulium sp. iridescent virus]

RecName: Full=Major capsid protein; Short=MCP; AltName: Full=P50 [Invertebrate iridescent virus 6]
RecName: Full=Major capsid protein; Short=MCP; AltName: Full=P50 [Wiseana iridescent virus]
RecName: Full=Major capsid protein; Short=MCP; AltName: Full=P50 [Costelytra zealandica iridescent virus]
RecName: Full=Major capsid protein; Short=MCP [Invertebrate iridescent virus 3]
RecName: Full=Major capsid protein; Short=MCP; AltName: Full=P50 [Simulium sp. irid

In [ ]:
descs = record.alignments[0].hit_def.split(" >")

target = None
for desc in descs:
    if desc.find("partial") == -1:
        target = desc
        break

if target:
    print("Found sequence accession number!")
    print("Accession:", target.split("|")[1])
else:
    print("404 not found!")

Found sequence accession number!
Accession: YP_010084900.1
